In [1]:
!sudo -H pip install -q transformers --upgrade

In [2]:
!sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

     |████████████████████████████████| 20.0MB 6.5MB/s eta 0:00:01
     |████████████████████████████████| 184kB 18.7MB/s eta 0:00:01
     |████████████████████████████████| 389kB 108.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 41.1MB/s  eta 0:00:01
     |████████████████████████████████| 112kB 119.5MB/s eta 0:00:01


     |████████████████████████████████| 6.2MB 99.9MB/s 


     |████████████████████████████████| 225kB 105.2MB/s eta 0:00:01
     |████████████████████████████████| 235kB 114.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 114.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 44.4MB/s  eta 0:00:01
     |████████████████████████████████| 235kB 112.7MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 103.3MB/s eta 0:00:01


     |████████████████████████████████| 389kB 108.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 43.3MB/s  eta 0:00:01
     |████████████████████████████████| 3.8MB 101.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 113.7MB/s eta 0:00:01
     |████████████████████████████████| 51kB 29.7MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 92.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 35.7MB/s eta 0:00:01
     |████████████████████████████████| 471kB 101.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 41.3MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 36.7MB/s eta 0:00:01
  Created wheel for sqlitedict: filename=sqlitedict-2.1.0-cp38-none-any.whl size=16865 sha256=81ca72d189c6320264a96f9ceb96c6ad8f95ab52c2a693ffa86a00ddfaa35fa5
  Stored in directory: /root/.cache/pip/wheels/2e/10/02/9725042cf8a987b45e38bcfb106f4b0cdcc9d8968e248b93e1
  Created wheel for rouge-score: filename=rouge_score-0.1.2-cp3

  Found existing installation: platformdirs 3.6.0
    Uninstalling platformdirs-3.6.0:
      Successfully uninstalled platformdirs-3.6.0
  Found existing installation: pluggy 1.0.0
    Uninstalling pluggy-1.0.0:
      Successfully uninstalled pluggy-1.0.0


In [1]:
import pandas

In [2]:
import numpy

In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

In [4]:
!pip list | grep cuda

nvidia-cuda-cupti-cu11   11.7.101   
nvidia-cuda-nvrtc-cu11   11.7.99    
nvidia-cuda-runtime-cu11 11.7.99    


In [5]:
import pandas

In [6]:
import torch
torch.__version__

'2.0.1+cu117'

In [7]:
model_path = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'

In [8]:
quant_path = '/data/mistral/query-to-mql/exp-10/nov-20/quant-model'

In [21]:
quant_config = {"zero_point":True,
               "q_group_size": 1024,
               "w_bit": 8,
               "version": "GEMM"
               }

In [11]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True, device_map='auto')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [22]:
#Quantize
# dont run it again
#model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.


OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB (GPU 0; 15.60 GiB total capacity; 14.75 GiB already allocated; 120.94 MiB free; 15.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
#model.save_quantized(quant_path)
#tokenizer.save_pretrained(quant_path)

('/data/quantization-trials/quant-model/tokenizer_config.json',
 '/data/quantization-trials/quant-model/special_tokens_map.json',
 '/data/quantization-trials/quant-model/vocab.json',
 '/data/quantization-trials/quant-model/merges.txt',
 '/data/quantization-trials/quant-model/added_tokens.json',
 '/data/quantization-trials/quant-model/tokenizer.json')

### Inference on quantized model

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/data/quantization-trials/quant-model'
model_path = '/data/quantization-trials/merged-model'

In [14]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [15]:
import time

In [23]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is art")
print("time taken is :", time.time()-start)

time taken is : 21.147445678710938


In [24]:
print(output1)

</s>what is art?

Art is a way of expressing one's self. It is a way of expressing one's thoughts, feelings, and ideas. It is a way of expressing one's self through the use of various media.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art for

### Inference on un-quantized model

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
model_path = '/data/quantization-trials/merged-model'

In [20]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [25]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is art")
print("time taken is :", time.time()-start)

time taken is : 23.784337282180786


In [26]:
print(output2)

</s>what is art?

Art is a creative expression of the human mind. It is a way of expressing the human mind through the use of various media.

The word art is derived from the Latin word artus, which means "to make". The word art is used in many different contexts, including:

- to describe the beauty of a work of art

- to describe the meaning of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work 

In [26]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [27]:
quant_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (v_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (q_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (out_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): WQLinear_GEMM(in_features=2048, out